In [1]:
import tensorflow as tf

In [5]:
base_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights="imagenet", input_shape=(
            256, 256, 3), pooling='avg')

219070464/219055592 [==============================] - 7s 0us/step


In [7]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, w_init, name='custom_layer'):
        super(CustomLayer, self).__init__(name=name)
        units = w_init.shape[1]
        self.w = tf.Variable(
            initial_value=w_init,
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        embeddings = tf.math.l2_normalize(inputs, axis=1, epsilon=1e-10)  # b x 2048, w: 2048 x 17
        t = tf.matmul(embeddings, self.w)                                # b x 17
        t = tf.math.l2_normalize(t, axis=1, epsilon=1e-10)
        z = t + self.b
        return tf.keras.activations.softmax(z, axis=-1)

In [3]:
inputs = tf.keras.Input(shape=(256, 256, 3))
x = base_model(inputs, training=False)